In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm as tq
import nltk
import time
import warnings
from sklearn.feature_extraction.text import  TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import log_loss,accuracy_score,classification_report,confusion_matrix

from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings('ignore')

C:\Users\Danish\AppData\Roaming\Python\Python37\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
# glove = open(r'Z:\DS DATA\glove.6B.300d.txt',encoding="utf8")

# start_time =time.clock()
# glovedict={}
# for line in glove:
#     l=line.split()
#     glovedict[l[0]] =np.asarray(l[1:])
# print(time.clock()-start_time,"seconds")        

In [59]:
df = pd.read_csv(r"Z:\DS DATA\feature_train.csv")
print(df.shape)
print(list(df.columns))

(404290, 33)
['id', 'qid1', 'qid2', 'question1', 'question2', 'is_duplicate', 'freq_qid1', 'freq_qid2', 'q1_total_words', 'q2_total_words', 'q1_n_words', 'q2_n_words', 'q1_union_q2', 'q1_q2_word_share', 'total_words', 'word_share', 'intr_by_union', 'fq1_pls_fq2', 'fq1_diff_fq2', 'cwc_min', 'cwc_max', 'csc_min', 'csc_max', 'ctc_min', 'ctc_max', 'fw_com', 'lw_com', 'abs_diff', 'ratio', 'token_set_ratio', 'token_sort_ratio', 'fuzz_ratio', 'partial_ratio']


In [60]:
Target = df["is_duplicate"]
df.dropna(axis=0,inplace=True)
df.drop(columns=['id','qid1','qid2','is_duplicate'],axis=1,inplace=True)
print(df.shape)
df.head(2)

(404210, 29)


,question1,question2,freq_qid1,freq_qid2,q1_total_words,q2_total_words,q1_n_words,q2_n_words,q1_union_q2,q1_q2_word_share,...,ctc_min,ctc_max,fw_com,lw_com,abs_diff,ratio,token_set_ratio,token_sort_ratio,fuzz_ratio,partial_ratio
0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...,1,1,14,12,12,11,13,10,...,0.916659,0.785709,1.0,0.0,2.0,13.0,100,93,93,100
1,what is the story of kohinoor koh-i-noor dia...,what would happen if the indian government sto...,4,1,8,13,8,12,16,4,...,0.624992,0.384612,1.0,0.0,5.0,10.5,86,63,66,75


In [31]:
# Initilizing TFIDF VECTORIZEER FOR QUESTION 1 AND QUESTION 2
Tfidf_Vect_question1 = TfidfVectorizer()
Tfidf_Vect_question2 = TfidfVectorizer()


# Creating features of question
Tfidf_q1 = Tfidf_Vect_question1.fit_transform(df["question1"].values.astype("U")) 
Tfidf_q2 = Tfidf_Vect_question2.fit_transform(df["question2"].values.astype("U")) 

In [32]:
print("Total no of features created from quesiton1 are:  ",len(Tfidf_Vect_question1.get_feature_names()))
print("Total no of features created from quesiton2 are:  ",len(Tfidf_Vect_question2.get_feature_names()))

Total no of features created from quesiton1 are:   62092
Total no of features created from quesiton2 are:   57137


In [54]:
from scipy.sparse import hstack

# scipy.sparse.hstack(blocks, format=None, dtype=None)
    # Stack sparse matrices horizontally (column wise)

# Parameters
    # blocks
    # sequence of sparse matrices with compatible shapes

# formatstr
    # sparse format of the result (e.g., “csr”) by default an appropriate
    # sparse matrix format is returned. This choice is subject to change.

# dtypedtype, optional
    # The data-type of the output matrix. If not given, the dtype is determined from that of


# from scipy.sparse import hstack,coo_matrix
# A=coo_matrix([[1,2],[3,5]])
# B=coo_matrix([[0],[4]])
# d=hstack([A,B]).toarray()
# d
# ----------------output--------------
# array([[1, 2, 0],
#        [3, 5, 4]], dtype=int32)
question1_question2 = hstack((Tfidf_q1,Tfidf_q2))

In [61]:
df.drop(columns=['question1','question2'],inplace=True)

In [67]:
df_features_tfidf_features = hstack((df,question1_question2),format="csr",dtype="float64")
print(df_features_tfidf_features.get_shape()) # also shape can be used to and works same 

(404210, 119256)


In [3]:
df2=df[:][:100000]
# docs = list(df2['question1']) + list(df2['question2']) 
print(df2["is_duplicate"].value_counts())
# print("percentage wise ",
#       ((df2["is_duplicate"].value_counts()[0])/len(df2["is_duplicate"])*100),"%"
#       "\t",((df2["is_duplicate"].value_counts()[1])/len(df2["is_duplicate"])*100),"%")

0    62746
1    37254
Name: is_duplicate, dtype: int64


In [4]:
print(((df2["is_duplicate"].value_counts()[0])/len(df2["is_duplicate"])))
print(((df2["is_duplicate"].value_counts()[1])/len(df2["is_duplicate"])))


0.62746
0.37254


In [6]:
df2.fillna(value=" ",inplace=True)

In [7]:
from nltk.tokenize import word_tokenize
HEIGHEST_LEN = 0
for i in range(len(df2['question1'])):
        HEIGHEST_LEN=max(HEIGHEST_LEN,len(word_tokenize(df2['question1'][i])))
        HEIGHEST_LEN=max(HEIGHEST_LEN,len(word_tokenize(df2['question2'][i])))
#     print(len(word_tokenize(df2['question1'][i])),len(word_tokenize(df2['question2'][i])))    
HEIGHEST_LEN


253

In [8]:
docs = list(df2['question1'].tolist() +df2['question2'].tolist())

In [9]:
start_time =time.clock()
tfidf = TfidfVectorizer()
# countvect =CountVectorizer()
tfidf.fit_transform(docs)
tfidf_dict = dict(zip(tfidf.get_feature_names(),tfidf.idf_))
print(time.clock()-start_time,"seconds")
# len(word2vec)

3.370890000000003 seconds


In [10]:
from nltk.tokenize import word_tokenize

start_time =time.clock()
def get_question_tfidf(x):
    x_np=np.zeros(HEIGHEST_LEN)    
    q1=word_tokenize(x)
    for i in range(len(q1)):
        x_np[i] = tfidf_dict.get(q1[i],0)
    return x_np
q1_x = df2['question1'].apply(get_question_tfidf)
q2_x = df2['question2'].apply(get_question_tfidf)
print(time.clock()-start_time,"seconds")       

29.9502947 seconds


In [11]:
def Create_list(series):
    series_list=[]
    for i in range(len(series)):
        series_list.append(series[i])
    return series_list


def Make_columns(names,par):
    columns_names = []
    for i in range(1,names+1):
        columns_names.append(str(i)+par)
    return columns_names

In [12]:
temp_df1 = pd.DataFrame(data=Create_list(q1_x),columns=Make_columns(HEIGHEST_LEN,"_x"))
temp_df2 = pd.DataFrame(data=Create_list(q2_x),columns=Make_columns(HEIGHEST_LEN,"_y"))


In [13]:
alpha = [10 ** x for x in range(-5, 2)]
alpha

[1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10]

In [14]:
final_df = pd.concat([df2,temp_df1,temp_df2],axis=1)

In [15]:
print(df2.shape)
print(temp_df1.shape)
print(temp_df2.shape)
print(final_df.shape)

(100000, 33)
(100000, 253)
(100000, 253)
(100000, 539)


In [587]:
# import sqlite3
# conn = sqlite3.connect("Any_Database_Name.db")

# final_df.to_sql('Some_Table_Name', conn)

# sql_string = 'SELECT * FROM Some_Table_Name'
# df33 = pd.read_sql(sql_string, conn)

In [549]:
y=final_df["is_duplicate"]
final_df.drop(columns=['question1','question2',"is_duplicate","qid1","qid2"],inplace=True)

In [432]:
# df2.to_csv(r"Z:\DS DATA\data.csv",index=False)

In [550]:
from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test = train_test_split(
    final_df, y, stratify=y, test_size=0.2)

In [551]:
print(type(X_train),X_train.shape,"  X_train")
print(type(X_test),X_test.shape,"  X_test")
print(type(y_train),y_train.shape,"  y_train")
print(type(y_test),y_test.shape,"  y_test")

<class 'pandas.core.frame.DataFrame'> (8000, 492)   X_train
<class 'pandas.core.frame.DataFrame'> (2000, 492)   X_test
<class 'pandas.core.series.Series'> (8000,)   y_train
<class 'pandas.core.series.Series'> (2000,)   y_test


<h1>Building a random model to calculate the worest-case of log_loss</h1>

In [552]:
# this is line creates an ndarray of shape 3000x2 
pred_y = np.zeros((len(y_test),2))

for i in range(len(y_test)):
#     this is line will create a array 1X2 
#     with random distribution populated in it
    rand_probs = np.random.rand(1,2)
    pred_y[i]= ((rand_probs/sum(sum(rand_probs)))[0])
print(log_loss(y_test,pred_y,eps=1e-15))

In [47]:
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
predected = clf.predict(X_test)
print(log_loss(y_test,predected,eps=1e-15))

9.99200722162641e-16


In [50]:
# print(list(y_test[:100]))
clf.score(X_test, y_test)


1.0

In [9]:
# SOURCE PDF DOWNLOADED

# df_x = pd.read_csv(r"Z:\DS DATA\train.csv")

# tfidfvectorizer_q1 = TfidfVectorizer()
# q1_tfidf = tfidfvectorizer_q1.fit_transform(df_x["question1"].values.astype("U"))
# q2_tfidf = tfidfvectorizer_q1.fit_transform(df_x["question2"].values.astype("U"))

# from scipy.sparse import hstack
# q1_q2 = hstack((q1_tfidf,q2_tfidf))

# df_x1 = pd.read_csv(r"Z:\DS DATA\feature_train.csv")

# df_x1.drop(columns=["id","qid1","qid2","question1","question2"],inplace=True)

# target = df_x1["is_duplicate"]

# df_tfidf = hstack((df_x1,q1_q2),format="csr",dtype="float64")

# from sklearn.model_selection import train_test_split

# X_train,X_test, y_train, y_test = train_test_split(
#     df_tfidf, target, stratify=target, test_size=0.3)

# print(type(X_train),X_train.shape,"  X_train")
# print(type(X_test),X_test.shape,"  X_test")
# print(type(y_train),y_train.shape,"  y_train")
# print(type(y_test),y_test.shape,"  y_test")

# clf = LogisticRegression(random_state=0).fit(X_train, y_train)
# predected = clf.predict(X_test)
# print(log_loss(y_test,predected,eps=1e-15))